In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [3]:
inputs=torch.Tensor(1, 1,28, 28)
inputs

tensor([[[[ 0.0000e+00,  0.0000e+00,  1.4396e-38,  0.0000e+00,  4.4870e-42,
            0.0000e+00,         nan,         nan,  6.7914e-33,  4.0104e+18,
            4.2824e-42,  3.1809e-43,  0.0000e+00,  0.0000e+00,  1.4349e-42,
            2.2959e-41,  8.8575e-39,  9.1835e-39,  9.9187e-39,  4.0041e-38,
            9.1836e-39,  1.7339e-37,  6.4068e-37,  9.1836e-39,  2.7743e-36,
            9.1851e-39,  1.1097e-35,  9.1851e-39],
          [ 4.4389e-35,  2.6805e-36,  7.1025e-34,  2.6245e-33,  2.6801e-36,
            1.1364e-32,  4.1994e-32,  1.6798e-31,  6.5203e-39,  6.8620e-34,
            6.8612e-34,  7.7016e-37,  6.7019e-37,  2.0343e-33,  3.2650e-43,
           -9.0888e+15,  1.8318e+25,  2.6381e+23,  7.2076e+31,  5.2943e-14,
            5.4826e+05,  6.8291e+22,  1.7553e-40,  3.1040e-38,  1.6173e+25,
            1.8524e+28,  2.8175e+20,  5.4267e-14],
          [ 1.0393e+22,  1.1532e+27,  1.6457e+19, -1.6638e+16,  6.8589e+22,
            1.6881e+25,  1.0836e+24,  7.9316e+34,  1.3873e-43,

In [4]:
conv1=nn.Conv2d(1, 32, 3, stride=1, padding=1)
pool=nn.MaxPool2d(2)
conv2=nn.Conv2d(32,64, 3, stride=1, padding=1)
out=conv1(inputs)
out.shape
out=pool(out)
out.shape
out=conv2(out)
out.shape
out=pool(out)
out.shape
out=out.view(out.size(0), -1)
out.shape
fc=nn.Linear(3136, 10)
out=fc(out)
out.shape

torch.Size([1, 10])

In [2]:
mnist_train=dsets.MNIST(root='/MNIST_data/', train=True, transform=transforms.ToTensor(), download=True)
mnist_test=dsets.MNIST(root='/MNIST_data/', train=False, transform=transforms.ToTensor(), download=True)



Extracting /MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to /MNIST_data/MNIST/raw



Extracting /MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to /MNIST_data/MNIST/raw



Extracting /MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /MNIST_data/MNIST/raw



Extracting /MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /MNIST_data/MNIST/raw
Processing...
Done!


In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [0]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [0]:
learning_rate=0.001
training_epochs=15
batch_size=100

In [0]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
    
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)  
        out = self.fc(out)
        return out

In [0]:
model=CNN().to(device)

In [0]:
criterion=nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [30]:
total_batch=len(data_loader)
for epoch in range(training_epochs):
  avg_cost=0

  for X, Y in data_loader:
    X=X.to(device)
    Y=Y.to(device)
    optimizer.zero_grad()
    hypothesis=model(X)

    cost=criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost+=cost/total_batch

  print('[Epoch:{}] cost= {}'.format(epoch+1, avg_cost))

print('Learning Finished')

[Epoch:1] cost= 0.22396428883075714
[Epoch:2] cost= 0.062215764075517654
[Epoch:3] cost= 0.044817350804805756
[Epoch:4] cost= 0.03552622348070145
[Epoch:5] cost= 0.02898259647190571
[Epoch:6] cost= 0.02484794706106186
[Epoch:7] cost= 0.02070016600191593
[Epoch:8] cost= 0.018210332840681076
[Epoch:9] cost= 0.01506195217370987
[Epoch:10] cost= 0.012585562653839588
[Epoch:11] cost= 0.009993097744882107
[Epoch:12] cost= 0.010541776195168495
[Epoch:13] cost= 0.0085977828130126
[Epoch:14] cost= 0.007436060346662998
[Epoch:15] cost= 0.005328935571014881
Learning Finished


In [31]:
with torch.no_grad():
  X_test=mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y_test=mnist_test.test_labels.to(device)

  prediction=model(X_test)
  correct_prediction=torch.argmax(prediction, 1)==Y_test
  accuracy=correct_prediction.float().mean()
  print('Accuracy', accuracy.item())


Accuracy 0.9870999455451965


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [0]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
    
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3=torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1,padding=1),
            torch.nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc1 = nn.Linear(3*3*128, 625)
        self.relu=nn.ReLU()
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)  
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [41]:
model=CNN().to(device)

value=torch.Tensor(1, 1, 28, 28).to(device)
print((model(value)).shape)

torch.Size([1, 10])


In [0]:
criterion=nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [43]:
total_batch=len(data_loader)
for epoch in range(training_epochs):
  avg_cost=0

  for X, Y in data_loader:
    X=X.to(device)
    Y=Y.to(device)
    optimizer.zero_grad()
    hypothesis=model(X)

    cost=criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost+=cost/total_batch

  print('[Epoch:{}] cost= {}'.format(epoch+1, avg_cost))

print('Learning Finished')

[Epoch:1] cost= 0.1616082787513733
[Epoch:2] cost= 0.04166105017066002
[Epoch:3] cost= 0.031178105622529984
[Epoch:4] cost= 0.022192517295479774
[Epoch:5] cost= 0.01759803108870983
[Epoch:6] cost= 0.0147355105727911
[Epoch:7] cost= 0.012392195872962475
[Epoch:8] cost= 0.009382558986544609
[Epoch:9] cost= 0.010531286709010601
[Epoch:10] cost= 0.007864768616855145
[Epoch:11] cost= 0.007288792636245489
[Epoch:12] cost= 0.00773099297657609
[Epoch:13] cost= 0.004898736719042063
[Epoch:14] cost= 0.006648554466664791
[Epoch:15] cost= 0.005461052991449833
Learning Finished


In [44]:
with torch.no_grad():
  X_test=mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y_test=mnist_test.test_labels.to(device)

  prediction=model(X_test)
  correct_prediction=torch.argmax(prediction, 1)==Y_test
  accuracy=correct_prediction.float().mean()
  print('Accuracy', accuracy.item())

Accuracy 0.9908999800682068


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
